In [ ]:
from pathlib import Path

from fractopo import Network
from fractopo.general import read_geofile, JOBLIB_CACHE
from fractopo.analysis.length_distributions import determine_fit, calculate_exponent
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from multi_network_analysis import multi_network_analysis

In [ ]:
trace_lengths_1_10 = pd.read_csv("../outputs/networks/1_10/trace_lengths.csv")[
    "lengths"
].values
trace_lengths_1_20k = pd.read_csv("../outputs/networks/1_20000/trace_lengths.csv")[
    "lengths"
].values
trace_lengths_1_200k = pd.read_csv(
    "../outputs/networks/1_200000_int/trace_lengths.csv"
)["lengths"].values

In [ ]:
@JOBLIB_CACHE.cache
def resolve_censoring_fits(lengths, num=20):
    censoring_cut_offs = np.linspace(
        start=lengths.max() + 0.001, stop=lengths.min(), num=num
    )
    fits = []

    for censoring_cut_off in censoring_cut_offs:
        censor_cut_off_lengths = lengths[lengths < censoring_cut_off]
        fit = determine_fit(censor_cut_off_lengths)
        fits.append(fit)

    return fits, censoring_cut_offs

In [ ]:
fits_1_10, zc = resolve_censoring_fits(trace_lengths_1_10)
fits_1_20k, censoring_cut_offs_1_20k = resolve_censoring_fits(trace_lengths_1_20k)
fits_1_200k, censoring_cut_offs_1_200k = resolve_censoring_fits(trace_lengths_1_200k)

In [ ]:
def visualize_effect_of_censoring(censoring_cut_offs, fits, suptitle, lengths):

    exponents = [calculate_exponent(fit.alpha) for fit in fits]
    cut_offs = [fit.xmin for fit in fits]

    def _resolve_mask(lengths, censoring_cut_off, cut_off):
        lengths_below_censoring = lenghts < censoring_cut_off
        lengths_above_cut_off = lengths > cut_off
        return lengths_below_censoring & lengths_above_cut_off

    cut_off_proportions = [
        1 - (sum(fit.data > fit.xmin) / len(lengths)) for fit in fits
    ]
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    ax_1 = axes[0]
    ax_1.scatter(x=censoring_cut_offs, y=exponents)
    ax_1.set_title("Censoring cut-off vs. power-law exponent")

    ax_2 = axes[1]
    ax_2.scatter(x=censoring_cut_offs, y=cut_offs)
    ax_2.set_title("Censoring cut-off vs. truncation cut-off")

    ax_3 = axes[2]
    ax_3.scatter(x=censoring_cut_offs, y=cut_off_proportions)
    ax_3.set_title("Censoring cut-off vs. cut off proportions")

    fig.suptitle(suptitle)
    return fig, axes

In [ ]:
for fits, suptitle, censoring_cut_offs, lengths in zip(
    [fits_1_10, fits_1_20k, fits_1_200k],
    ["1:10", "1:20k", "1:200k"],
    [censoring_cut_offs_1_10, censoring_cut_offs_1_20k, censoring_cut_offs_1_200k],
    [trace_lengths_1_10, trace_lengths_1_20k, trace_lengths_1_200k],
):
    fig, axes = visualize_effect_of_censoring(
        fits=fits,
        suptitle=suptitle,
        censoring_cut_offs=censoring_cut_offs,
        lengths=lengths,
    )